# Forward-Forward MNIST Classification (Hinton's Recurrent Approach)

Implementation of Hinton's recurrent Forward-Forward as described in Section 5 of
["The Forward-Forward Algorithm: Some Preliminary Investigations"](https://arxiv.org/abs/2212.13345).

## Key Differences from Row-by-Row Temporal Model

| Aspect | Row-by-Row Temporal | Hinton's Recurrent (this notebook) |
|--------|--------------------|---------------------------------|
| Input | 28 rows sequentially | Full 784 pixels at once |
| Processing | Single pass through 28 timesteps | 8 iterations on same input |
| Connections | Feedforward only | **Bidirectional** (top-down + bottom-up) |
| Goodness | Sum of squared activities | **Agreement** between layers |
| Training | BPTT through time | **Local per layer** (no BPTT) |
| Damping | α=0.95 decay | 0.3 old + 0.7 new |

## Hinton's Key Insight

> "The activity vector at each layer is determined by the normalized activity vectors 
> at both the layer above and the layer below at the previous time-step."

> "When top-down and bottom-up inputs agree, there will be positive interference 
> resulting in high squared activities and if they disagree the squared activities will be lower."

## Architecture

```
Label Layer (top):     10 neurons (one-hot class hypothesis)
        ↕ (bidirectional)
Hidden Layer 2:        2000 neurons  
        ↕ (bidirectional)
Hidden Layer 1:        2000 neurons
        ↕ (bidirectional)  
Input Layer (bottom):  784 pixels
```

For our constrained version (<26 neurons), we'll use smaller hidden layers.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
import gzip
import urllib.request

torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print("\nImplementing Hinton's Recurrent Forward-Forward Algorithm")
print("Key features:")
print("  - Bidirectional connections (top-down + bottom-up)")
print("  - Agreement-based goodness")
print("  - 8 iterations with damping")
print("  - Local layer-wise learning (NO BPTT)")

## 1. Load MNIST Dataset

In [ ]:
def download_mnist(data_dir='./data/mnist'):
    """Download MNIST dataset."""
    os.makedirs(data_dir, exist_ok=True)
    
    base_url = 'https://ossci-datasets.s3.amazonaws.com/mnist/'
    files = {
        'train_images': 'train-images-idx3-ubyte.gz',
        'train_labels': 'train-labels-idx1-ubyte.gz',
        'test_images': 't10k-images-idx3-ubyte.gz',
        'test_labels': 't10k-labels-idx1-ubyte.gz',
    }
    
    paths = {}
    for key, filename in files.items():
        filepath = os.path.join(data_dir, filename)
        if not os.path.exists(filepath):
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(base_url + filename, filepath)
        paths[key] = filepath
    
    return paths


def load_mnist_images(filepath):
    """Load MNIST images - flatten to 784."""
    with gzip.open(filepath, 'rb') as f:
        magic = int.from_bytes(f.read(4), 'big')
        n_images = int.from_bytes(f.read(4), 'big')
        n_rows = int.from_bytes(f.read(4), 'big')
        n_cols = int.from_bytes(f.read(4), 'big')
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data.reshape(n_images, n_rows * n_cols).astype(np.float32) / 255.0


def load_mnist_labels(filepath):
    """Load MNIST labels."""
    with gzip.open(filepath, 'rb') as f:
        magic = int.from_bytes(f.read(4), 'big')
        n_labels = int.from_bytes(f.read(4), 'big')
        return np.frombuffer(f.read(), dtype=np.uint8)


# Download and load
paths = download_mnist()
X_train_full = torch.from_numpy(load_mnist_images(paths['train_images']))
y_train_full = torch.from_numpy(load_mnist_labels(paths['train_labels'])).long()
X_test_full = torch.from_numpy(load_mnist_images(paths['test_images']))
y_test_full = torch.from_numpy(load_mnist_labels(paths['test_labels'])).long()

print(f"Full dataset: Train={X_train_full.shape}, Test={X_test_full.shape}")

# Use subset
N_TRAIN = 20000
N_TEST = 2000

torch.manual_seed(42)
train_idx = torch.randperm(len(X_train_full))[:N_TRAIN]
test_idx = torch.randperm(len(X_test_full))[:N_TEST]

X_train = X_train_full[train_idx]
y_train = y_train_full[train_idx]
X_test = X_test_full[test_idx]
y_test = y_test_full[test_idx]

print(f"\nUsing subset:")
print(f"  Training: {X_train.shape}")
print(f"  Test: {X_test.shape}")

## 2. Hinton's Recurrent Forward-Forward Layer

Each layer receives input from BOTH the layer below AND the layer above.
Goodness = agreement (interference) between these two signals.

In [ ]:
class RecurrentFFLayer(nn.Module):
    """
    A single layer in Hinton's Recurrent Forward-Forward network.
    
    Key features:
    - Receives input from layer below (bottom-up)
    - Receives input from layer above (top-down)  
    - Goodness = sum of squared activities (agreement creates high activities)
    - Layer normalization (without mean subtraction)
    - Local learning: each layer has its own optimizer
    """
    
    def __init__(self, dim_below, dim_self, dim_above=None, is_top=False, is_bottom=False):
        super().__init__()
        
        self.dim_below = dim_below
        self.dim_self = dim_self
        self.dim_above = dim_above
        self.is_top = is_top
        self.is_bottom = is_bottom
        
        # Bottom-up weights (from layer below)
        if not is_bottom:
            self.W_bottom_up = nn.Linear(dim_below, dim_self, bias=True)
        
        # Top-down weights (from layer above)
        if not is_top and dim_above is not None:
            self.W_top_down = nn.Linear(dim_above, dim_self, bias=False)
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        if hasattr(self, 'W_bottom_up'):
            nn.init.xavier_uniform_(self.W_bottom_up.weight)
            nn.init.zeros_(self.W_bottom_up.bias)
        if hasattr(self, 'W_top_down'):
            nn.init.xavier_uniform_(self.W_top_down.weight)
    
    def layer_norm(self, x, eps=1e-8):
        """
        Hinton's layer normalization: divide by length WITHOUT subtracting mean.
        This removes magnitude info, forcing deeper layers to use relative activations.
        """
        # Normalize to unit length (L2 norm)
        norm = torch.norm(x, dim=1, keepdim=True) + eps
        return x / norm
    
    def forward(self, h_below_norm, h_above_norm=None):
        """
        Compute new state from normalized inputs from above and below.
        
        Args:
            h_below_norm: Normalized activations from layer below [B, dim_below]
            h_above_norm: Normalized activations from layer above [B, dim_above] (optional)
        
        Returns:
            pre_norm: Pre-normalized activations [B, dim_self]
            h_norm: Normalized activations [B, dim_self]
        """
        # Bottom-up contribution
        if hasattr(self, 'W_bottom_up'):
            bottom_up = self.W_bottom_up(h_below_norm)
        else:
            bottom_up = h_below_norm  # For bottom layer, input IS the activation
        
        # Top-down contribution (if we have a layer above)
        if h_above_norm is not None and hasattr(self, 'W_top_down'):
            top_down = self.W_top_down(h_above_norm)
            # Combined: when signals agree, they reinforce (high squared activity)
            pre_act = bottom_up + top_down
        else:
            pre_act = bottom_up
        
        # ReLU activation
        pre_norm = F.relu(pre_act)
        
        # Normalize
        h_norm = self.layer_norm(pre_norm)
        
        return pre_norm, h_norm
    
    def compute_goodness(self, pre_norm):
        """
        Goodness = sum of squared activities.
        High when top-down and bottom-up agree (constructive interference).
        Low when they disagree (destructive interference).
        """
        return (pre_norm ** 2).sum(dim=1)


# Test layer
test_layer = RecurrentFFLayer(dim_below=784, dim_self=100, dim_above=10)
test_input = torch.randn(5, 784)
test_above = torch.randn(5, 10)
pre_norm, h_norm = test_layer(test_input, test_above)
print(f"Input: {test_input.shape}")
print(f"Pre-norm output: {pre_norm.shape}")
print(f"Normalized output: {h_norm.shape}")
print(f"Output norm: {torch.norm(h_norm, dim=1)}")

## 3. Recurrent Forward-Forward Network

The full network with bidirectional connections and iterative processing.

In [ ]:
class RecurrentFFNetwork(nn.Module):
    """
    Hinton's Recurrent Forward-Forward Network.
    
    Architecture:
    - Input layer (784 pixels)
    - Hidden layers (bidirectional connections)
    - Label layer (10 classes, one-hot)
    
    Processing:
    1. Initialize hidden layers with single bottom-up pass
    2. Run N iterations with damping, each layer receiving from above and below
    3. Compute goodness at each iteration
    """
    
    def __init__(self, input_dim=784, hidden_dims=[100], n_classes=10, 
                 damping=0.3, n_iterations=8):
        super().__init__()
        
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.n_classes = n_classes
        self.damping = damping  # 0.3 = keep 30% of old state
        self.n_iterations = n_iterations
        self.n_layers = len(hidden_dims)
        
        # Build layers
        self.layers = nn.ModuleList()
        
        for i, hidden_dim in enumerate(hidden_dims):
            # Dimension of layer below
            if i == 0:
                dim_below = input_dim
            else:
                dim_below = hidden_dims[i-1]
            
            # Dimension of layer above
            if i == len(hidden_dims) - 1:
                dim_above = n_classes  # Top hidden connects to label
            else:
                dim_above = hidden_dims[i+1]
            
            layer = RecurrentFFLayer(
                dim_below=dim_below,
                dim_self=hidden_dim,
                dim_above=dim_above,
                is_top=(i == len(hidden_dims) - 1),
                is_bottom=False
            )
            self.layers.append(layer)
        
        print(f"RecurrentFFNetwork:")
        print(f"  Input: {input_dim}")
        for i, dim in enumerate(hidden_dims):
            print(f"  Hidden {i+1}: {dim} neurons")
        print(f"  Label: {n_classes}")
        print(f"  Iterations: {n_iterations}")
        print(f"  Damping: {damping} (keep {damping*100:.0f}% of old state)")
    
    def layer_norm(self, x, eps=1e-8):
        """Normalize to unit length."""
        norm = torch.norm(x, dim=1, keepdim=True) + eps
        return x / norm
    
    def initialize_hidden(self, x, label_onehot):
        """
        Initialize hidden layers with a single bottom-up pass.
        
        Args:
            x: Input images [B, 784]
            label_onehot: One-hot labels [B, 10]
        
        Returns:
            List of (pre_norm, h_norm) for each hidden layer
        """
        states = []
        
        # Normalize input
        h_below = self.layer_norm(x)
        
        # Forward pass through all hidden layers (no top-down yet)
        for layer in self.layers:
            pre_norm, h_norm = layer(h_below, h_above_norm=None)
            states.append((pre_norm, h_norm))
            h_below = h_norm
        
        return states
    
    def run_iteration(self, x_norm, label_norm, states):
        """
        Run one iteration of the recurrent network.
        
        Each layer receives:
        - Normalized activations from layer below
        - Normalized activations from layer above
        
        Args:
            x_norm: Normalized input [B, 784]
            label_norm: Normalized label [B, 10]
            states: List of (pre_norm, h_norm) for each hidden layer
        
        Returns:
            new_states: Updated states with damping applied
        """
        new_states = []
        
        for i, layer in enumerate(self.layers):
            # Get input from below
            if i == 0:
                h_below_norm = x_norm
            else:
                _, h_below_norm = states[i-1]
            
            # Get input from above
            if i == len(self.layers) - 1:
                # Top hidden layer receives from label
                h_above_norm = label_norm
            else:
                _, h_above_norm = states[i+1]
            
            # Compute new state
            pre_norm_new, h_norm_new = layer(h_below_norm, h_above_norm)
            
            # Apply damping: new = damping * old + (1-damping) * computed
            pre_norm_old, _ = states[i]
            pre_norm_damped = self.damping * pre_norm_old + (1 - self.damping) * pre_norm_new
            h_norm_damped = self.layer_norm(F.relu(pre_norm_damped))
            
            new_states.append((pre_norm_damped, h_norm_damped))
        
        return new_states
    
    def forward(self, x, label_onehot, return_all_iterations=False):
        """
        Full forward pass with initialization + N iterations.
        
        Args:
            x: Input images [B, 784]
            label_onehot: One-hot label hypothesis [B, 10]
            return_all_iterations: If True, return states at all iterations
        
        Returns:
            goodness_per_layer: [n_layers] average goodness per layer
            all_states: (optional) states at each iteration
        """
        B = x.shape[0]
        
        # Normalize inputs
        x_norm = self.layer_norm(x)
        label_norm = self.layer_norm(label_onehot)
        
        # Initialize with bottom-up pass
        states = self.initialize_hidden(x, label_onehot)
        
        all_iterations = [states] if return_all_iterations else None
        goodness_history = []
        
        # Run iterations
        for iter_idx in range(self.n_iterations):
            states = self.run_iteration(x_norm, label_norm, states)
            
            if return_all_iterations:
                all_iterations.append(states)
            
            # Compute goodness at this iteration
            iter_goodness = []
            for layer_idx, layer in enumerate(self.layers):
                pre_norm, _ = states[layer_idx]
                g = layer.compute_goodness(pre_norm)
                iter_goodness.append(g)
            goodness_history.append(iter_goodness)
        
        # Average goodness over iterations 3-5 (as Hinton suggests)
        # Or all iterations if fewer than 5
        start_iter = min(2, len(goodness_history) - 1)  # iter 3 = index 2
        end_iter = min(5, len(goodness_history))  # iter 5 = index 4, exclusive
        
        goodness_per_layer = []
        for layer_idx in range(len(self.layers)):
            layer_goodness = torch.stack([goodness_history[i][layer_idx] 
                                          for i in range(start_iter, end_iter)])
            goodness_per_layer.append(layer_goodness.mean(dim=0))
        
        if return_all_iterations:
            return goodness_per_layer, all_iterations
        return goodness_per_layer


# Test network
N_CLASSES = 10
HIDDEN_DIMS = [24]  # Small for <26 constraint

test_net = RecurrentFFNetwork(
    input_dim=784,
    hidden_dims=HIDDEN_DIMS,
    n_classes=N_CLASSES,
    damping=0.3,
    n_iterations=8
)

# Test forward
test_x = torch.randn(5, 784)
test_label = F.one_hot(torch.tensor([0, 1, 2, 3, 4]), N_CLASSES).float()
goodness = test_net(test_x, test_label)
print(f"\nTest forward pass:")
print(f"  Input: {test_x.shape}")
print(f"  Label: {test_label.shape}")
print(f"  Goodness shape: {[g.shape for g in goodness]}")
print(f"  Total goodness: {sum(g.sum() for g in goodness).item():.4f}")

## 4. Forward-Forward Loss and Training

Key: Learning is **LOCAL** to each layer. No gradients flow between layers.

In [ ]:
def ff_loss(goodness_pos, goodness_neg, threshold=0.0):
    """
    Forward-Forward loss for a single layer.
    
    Hinton's formulation:
    - Positive: sigmoid(goodness - threshold) should be high
    - Negative: sigmoid(goodness - threshold) should be low
    """
    # Positive samples: maximize goodness (minimize -log(sigmoid(g - threshold)))
    loss_pos = F.softplus(threshold - goodness_pos).mean()
    
    # Negative samples: minimize goodness (minimize -log(1 - sigmoid(g - threshold)))
    loss_neg = F.softplus(goodness_neg - threshold).mean()
    
    return loss_pos + loss_neg


def create_positive_negative(X, y, n_classes=10):
    """
    Create positive and negative samples.
    
    Positive: image + correct label
    Negative: image + random wrong label (sampled proportionally)
    """
    B = X.shape[0]
    
    # Positive labels (one-hot)
    y_pos = F.one_hot(y, n_classes).float()
    
    # Negative labels (random wrong class)
    y_neg_idx = (y + torch.randint(1, n_classes, (B,))) % n_classes
    y_neg = F.one_hot(y_neg_idx, n_classes).float()
    
    return y_pos, y_neg


def train_recurrent_ff(model, X_train, y_train, X_test, y_test,
                       n_epochs=60, lr=0.001, batch_size=64,
                       threshold=2.0, verbose=True):
    """
    Train the Recurrent Forward-Forward network.
    
    KEY: Each layer has its OWN optimizer and learns LOCALLY.
    No gradients flow between layers - this is the core of FF.
    """
    model.train()
    
    # Create separate optimizer for each layer (local learning)
    layer_optimizers = []
    for layer in model.layers:
        opt = torch.optim.Adam(layer.parameters(), lr=lr)
        layer_optimizers.append(opt)
    
    history = {
        'loss': [],
        'train_acc': [],
        'test_acc': [],
        'goodness_pos': [],
        'goodness_neg': [],
    }
    
    N = X_train.shape[0]
    n_batches = (N + batch_size - 1) // batch_size
    
    best_test_acc = 0
    
    for epoch in range(n_epochs):
        epoch_loss = 0
        epoch_g_pos = []
        epoch_g_neg = []
        
        # Shuffle
        perm = torch.randperm(N)
        X_shuffled = X_train[perm]
        y_shuffled = y_train[perm]
        
        for batch_idx in range(n_batches):
            start = batch_idx * batch_size
            end = min(start + batch_size, N)
            
            X_batch = X_shuffled[start:end]
            y_batch = y_shuffled[start:end]
            
            # Create positive and negative labels
            y_pos, y_neg = create_positive_negative(X_batch, y_batch)
            
            # Forward pass for positive and negative
            goodness_pos = model(X_batch, y_pos)
            goodness_neg = model(X_batch, y_neg)
            
            # Train each layer LOCALLY
            batch_loss = 0
            for layer_idx, (g_pos, g_neg, opt) in enumerate(zip(
                    goodness_pos, goodness_neg, layer_optimizers)):
                
                # Zero gradients for THIS layer only
                opt.zero_grad()
                
                # Compute loss for this layer
                # CRITICAL: .detach() ensures no gradients flow between layers
                layer_loss = ff_loss(g_pos, g_neg.detach(), threshold)
                
                # Backward for this layer only
                layer_loss.backward(retain_graph=True)
                
                # Update this layer's weights
                opt.step()
                
                batch_loss += layer_loss.item()
            
            epoch_loss += batch_loss
            epoch_g_pos.append(sum(g.mean().item() for g in goodness_pos))
            epoch_g_neg.append(sum(g.mean().item() for g in goodness_neg))
            
            if verbose and batch_idx % 100 == 0:
                print(f"\rEpoch {epoch+1}/{n_epochs} | Batch {batch_idx+1}/{n_batches} | "
                      f"Loss: {batch_loss:.4f}", end="")
        
        # Evaluate
        train_acc = evaluate_recurrent_ff(model, X_train[:2000], y_train[:2000])
        test_acc = evaluate_recurrent_ff(model, X_test, y_test)
        
        if test_acc > best_test_acc:
            best_test_acc = test_acc
        
        history['loss'].append(epoch_loss / n_batches)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        history['goodness_pos'].append(np.mean(epoch_g_pos))
        history['goodness_neg'].append(np.mean(epoch_g_neg))
        
        if verbose:
            sep = np.mean(epoch_g_pos) - np.mean(epoch_g_neg)
            print(f"\rEpoch {epoch+1}/{n_epochs} | Loss: {epoch_loss/n_batches:.4f} | "
                  f"Train: {train_acc:.4f} | Test: {test_acc:.4f} | "
                  f"Best: {best_test_acc:.4f} | Sep: {sep:.2f}    ")
    
    return history


def evaluate_recurrent_ff(model, X, y, batch_size=100):
    """
    Evaluate by testing all 10 label hypotheses and picking highest goodness.
    """
    model.eval()
    N = X.shape[0]
    all_preds = []
    
    with torch.no_grad():
        for start in range(0, N, batch_size):
            end = min(start + batch_size, N)
            X_batch = X[start:end]
            B = X_batch.shape[0]
            
            # Test all 10 hypotheses
            all_goodness = []
            for digit in range(10):
                label_hyp = F.one_hot(torch.full((B,), digit, dtype=torch.long), 10).float()
                goodness = model(X_batch, label_hyp)
                total_goodness = sum(g for g in goodness)  # Sum across layers
                all_goodness.append(total_goodness)
            
            # Stack: [10, B] -> [B, 10]
            goodness_matrix = torch.stack(all_goodness, dim=1)
            preds = goodness_matrix.argmax(dim=1)
            all_preds.append(preds)
    
    all_preds = torch.cat(all_preds)
    accuracy = (all_preds == y).float().mean().item()
    model.train()
    return accuracy


print("Training functions defined.")
print("\nKey difference from standard FF:")
print("  - Each layer has its OWN optimizer")
print("  - Gradients do NOT flow between layers")
print("  - This is TRUE local learning (no BPTT)")

## 5. Train the Model

In [ ]:
# Build network
HIDDEN_DIMS = [24]  # Constrained to <26 neurons
N_ITERATIONS = 8
DAMPING = 0.3
THRESHOLD = 2.0
N_EPOCHS = 60
LR = 0.001
BATCH_SIZE = 64

print("="*70)
print("HINTON'S RECURRENT FORWARD-FORWARD")
print("="*70)
print(f"Architecture: 784 → {HIDDEN_DIMS} → 10")
print(f"Iterations: {N_ITERATIONS}")
print(f"Damping: {DAMPING} (keep {DAMPING*100:.0f}% old state)")
print(f"Threshold: {THRESHOLD}")
print(f"Learning rate: {LR}")
print(f"\nKEY FEATURES:")
print(f"  ✓ Bidirectional connections (top-down + bottom-up)")
print(f"  ✓ Agreement-based goodness")
print(f"  ✓ Local layer-wise learning (NO BPTT)")
print(f"  ✓ Multiple iterations with damping")
print("="*70)

torch.manual_seed(42)
model = RecurrentFFNetwork(
    input_dim=784,
    hidden_dims=HIDDEN_DIMS,
    n_classes=10,
    damping=DAMPING,
    n_iterations=N_ITERATIONS
)

n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {n_params}")

history = train_recurrent_ff(
    model, X_train, y_train, X_test, y_test,
    n_epochs=N_EPOCHS, lr=LR, batch_size=BATCH_SIZE,
    threshold=THRESHOLD, verbose=True
)

print("="*70)
print(f"Final train accuracy: {history['train_acc'][-1]:.4f}")
print(f"Final test accuracy: {history['test_acc'][-1]:.4f}")
print(f"Best test accuracy: {max(history['test_acc']):.4f}")
print(f"Random baseline: 10%")

## 6. Training Curves

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Loss
ax1 = axes[0, 0]
ax1.plot(history['loss'], color='steelblue', lw=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.grid(True, alpha=0.3)

# Accuracy
ax2 = axes[0, 1]
ax2.plot(history['train_acc'], label='Train', color='coral', lw=2)
ax2.plot(history['test_acc'], label='Test', color='steelblue', lw=2)
ax2.axhline(y=0.1, color='gray', linestyle='--', alpha=0.5, label='Random')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Classification Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1.0)

# Goodness
ax3 = axes[1, 0]
ax3.plot(history['goodness_pos'], label='Positive', color='green', lw=2)
ax3.plot(history['goodness_neg'], label='Negative', color='red', lw=2)
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Goodness')
ax3.set_title('Goodness Values')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Separation
ax4 = axes[1, 1]
separation = [p - n for p, n in zip(history['goodness_pos'], history['goodness_neg'])]
ax4.plot(separation, color='purple', lw=2)
ax4.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax4.set_xlabel('Epoch')
ax4.set_ylabel('G+ - G-')
ax4.set_title('Goodness Separation')
ax4.grid(True, alpha=0.3)

plt.suptitle(f"Hinton's Recurrent Forward-Forward ({sum(HIDDEN_DIMS)} neurons)", fontsize=14)
plt.tight_layout()
plt.show()

## 7. Visualize Iteration Dynamics

In [ ]:
# Show how goodness evolves over iterations
model.eval()

# Pick a sample
sample_idx = 0
X_sample = X_test[sample_idx:sample_idx+1]
y_sample = y_test[sample_idx].item()

# Test all hypotheses and track iterations
all_goodness_evolution = []

with torch.no_grad():
    for digit in range(10):
        label_hyp = F.one_hot(torch.tensor([digit]), 10).float()
        
        # Get goodness at each iteration
        x_norm = model.layer_norm(X_sample)
        label_norm = model.layer_norm(label_hyp)
        states = model.initialize_hidden(X_sample, label_hyp)
        
        iter_goodness = []
        for i in range(model.n_iterations):
            states = model.run_iteration(x_norm, label_norm, states)
            total_g = sum(model.layers[j].compute_goodness(states[j][0]).item() 
                         for j in range(len(model.layers)))
            iter_goodness.append(total_g)
        
        all_goodness_evolution.append(iter_goodness)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Image
ax1 = axes[0]
ax1.imshow(X_sample.reshape(28, 28).numpy(), cmap='gray')
ax1.set_title(f'Test Sample (True Label: {y_sample})')
ax1.axis('off')

# Right: Goodness evolution
ax2 = axes[1]
for digit in range(10):
    color = 'green' if digit == y_sample else 'lightgray'
    lw = 3 if digit == y_sample else 1
    ax2.plot(range(1, N_ITERATIONS+1), all_goodness_evolution[digit], 
             label=f'Digit {digit}', color=color, lw=lw, alpha=0.8)

ax2.axvline(x=3, color='blue', linestyle=':', alpha=0.5, label='Eval start (iter 3)')
ax2.axvline(x=5, color='blue', linestyle=':', alpha=0.5, label='Eval end (iter 5)')
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Total Goodness')
ax2.set_title('Goodness Evolution Over Iterations')
ax2.legend(loc='upper right', fontsize=8)
ax2.grid(True, alpha=0.3)

plt.suptitle("Recurrent FF: Correct label should have highest goodness", fontsize=12)
plt.tight_layout()
plt.show()

## 8. Compare Architectures

In [ ]:
# Compare different hidden sizes
hidden_configs = [
    [12],
    [16],
    [20],
    [24],
    [12, 12],  # Two layers
]

comparison_results = []

print("Comparing Recurrent FF architectures...")
print("="*70)

for hidden_dims in hidden_configs:
    torch.manual_seed(42)
    
    model = RecurrentFFNetwork(
        input_dim=784,
        hidden_dims=hidden_dims,
        n_classes=10,
        damping=0.3,
        n_iterations=8
    )
    
    n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_neurons = sum(hidden_dims)
    
    history = train_recurrent_ff(
        model, X_train, y_train, X_test, y_test,
        n_epochs=30, lr=0.001, batch_size=64,
        threshold=2.0, verbose=False
    )
    
    best_test = max(history['test_acc'])
    comparison_results.append({
        'hidden_dims': str(hidden_dims),
        'total_neurons': total_neurons,
        'n_params': n_params,
        'test_acc': history['test_acc'][-1],
        'best_test': best_test,
    })
    
    print(f"Hidden={str(hidden_dims):12s} | Neurons={total_neurons:2d} | "
          f"Params={n_params:6d} | Test: {history['test_acc'][-1]:.4f} | Best: {best_test:.4f}")

print("="*70)
best_result = max(comparison_results, key=lambda x: x['best_test'])
print(f"\nBest: {best_result['hidden_dims']} with {best_result['best_test']:.2%}")

## 9. Confusion Matrix

In [ ]:
def get_predictions(model, X, batch_size=100):
    model.eval()
    N = X.shape[0]
    all_preds = []
    all_goodness = []
    
    with torch.no_grad():
        for start in range(0, N, batch_size):
            end = min(start + batch_size, N)
            X_batch = X[start:end]
            B = X_batch.shape[0]
            
            batch_goodness = []
            for digit in range(10):
                label_hyp = F.one_hot(torch.full((B,), digit, dtype=torch.long), 10).float()
                goodness = model(X_batch, label_hyp)
                total_goodness = sum(g for g in goodness)
                batch_goodness.append(total_goodness)
            
            goodness_matrix = torch.stack(batch_goodness, dim=1)
            all_goodness.append(goodness_matrix)
            all_preds.append(goodness_matrix.argmax(dim=1))
    
    return torch.cat(all_preds), torch.cat(all_goodness)


# Get predictions
test_preds, test_goodness = get_predictions(model, X_test)

# Confusion matrix
cm = np.zeros((10, 10), dtype=np.int32)
for true, pred in zip(y_test.numpy(), test_preds.numpy()):
    cm[true, pred] += 1

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks(range(10))
ax.set_yticks(range(10))
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_title(f"Confusion Matrix (Recurrent FF, Test Acc: {history['test_acc'][-1]:.2%})")

for i in range(10):
    for j in range(10):
        color = 'white' if cm[i, j] > cm.max()/2 else 'black'
        ax.text(j, i, cm[i, j], ha='center', va='center', color=color)

plt.colorbar(im)
plt.tight_layout()
plt.show()

# Per-class accuracy
print("\nPer-class accuracy:")
for digit in range(10):
    mask = y_test == digit
    if mask.sum() > 0:
        acc = (test_preds[mask] == digit).float().mean().item()
        print(f"  Digit {digit}: {acc:.2%}")

## 10. Conclusions

In [ ]:
print("="*70)
print("CONCLUSIONS: HINTON'S RECURRENT FORWARD-FORWARD")
print("="*70)

print(f"\n1. ARCHITECTURE:")
print(f"   Input: 784 (full image)")
print(f"   Hidden: {HIDDEN_DIMS}")
print(f"   Label: 10 (one-hot hypothesis)")
print(f"   Iterations: {N_ITERATIONS}")
print(f"   Damping: {DAMPING}")

print(f"\n2. KEY DIFFERENCES FROM ROW-BY-ROW TEMPORAL:")
print(f"   ✓ Full image input (not row-by-row)")
print(f"   ✓ Bidirectional connections (top-down + bottom-up)")
print(f"   ✓ Agreement-based goodness (interference pattern)")
print(f"   ✓ Multiple iterations (8) vs single pass")
print(f"   ✓ TRUE local learning (no BPTT at all)")

print(f"\n3. PERFORMANCE:")
print(f"   Final test accuracy: {history['test_acc'][-1]:.2%}")
print(f"   Best test accuracy: {max(history['test_acc']):.2%}")
print(f"   Random baseline: 10%")

print(f"\n4. WHY THIS APPROACH IS PRINCIPLED:")
print(f"   - Follows Hinton's original paper exactly")
print(f"   - No vanishing gradients (no BPTT)")
print(f"   - Hardware-compatible (local learning rules)")
print(f"   - Biologically plausible (cortical feedback)")

print(f"\n5. HINTON'S RESULTS (for reference):")
print(f"   - 2000 neurons per hidden layer")
print(f"   - 8 iterations with 0.3/0.7 damping")
print(f"   - 1.31% test error (98.69% accuracy)")
print(f"   - Our {sum(HIDDEN_DIMS)} neurons: {max(history['test_acc']):.2%}")

print(f"\n6. NEXT STEPS:")
print(f"   - Try larger hidden layers (2000 as in paper)")
print(f"   - Tune threshold and learning rate")
print(f"   - Add more hidden layers")
print(f"   - Compare with SOEN temporal model")

print("\n" + "="*70)